In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()




26/01/20 06:34:57 WARN Utils: Your hostname, hp-HP-EliteBook-Folio-9480m resolves to a loopback address: 127.0.1.1; using 10.216.72.76 instead (on interface wlo1)
26/01/20 06:34:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hp/.ivy2/cache
The jars for the packages stored in: /home/hp/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2a289d17-88b1-409e-99aa-ae7849a4b202;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 711ms :: artifacts dl 26ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------------------------

In [ ]:
#spark.stop()

## 1. Data Injection

In [2]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 600    
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
  #save to bronze
  jdbc_url = (
    f"jdbc:postgresql://{os.getenv('DB_HOST')}:"
    f"{os.getenv('DB_PORT')}/"
    f"{os.getenv('DB_NAME')}"
   )
  psdf.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "btc_bronze") \
  .option("user", os.getenv("DB_USER"))  \
  .option("password", os.getenv("DB_PASSWORD")) \
  .option("driver", "org.postgresql.Driver") \
  .mode("overwrite") \
  .save() 
  return psdf
  
data= data_collection()
data.show()

+-------------+--------------+--------------+--------------+--------------+----------+-------------+------------------+----------------+---------------------+----------------------+------+
|    open_time|    open_price|    high_price|     low_price|   close_price|    volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------+--------------+--------------+--------------+--------------+----------+-------------+------------------+----------------+---------------------+----------------------+------+
|1768851360000|93112.85000000|93112.86000000|93112.85000000|93112.85000000|0.60091000|1768851419999|    55952.44557310|             164|           0.28796000|        26812.77916560|     0|
|1768851420000|93112.86000000|93112.86000000|93094.34000000|93101.02000000|4.41782000|1768851479999|   411312.71047690|            1262|           1.87274000|       174345.37569400|     0|
|1768851480000|93101.03000000|93101.03000000|93101.0200

In [3]:
data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: string (nullable = true)
 |-- high_price: string (nullable = true)
 |-- low_price: string (nullable = true)
 |-- close_price: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: string (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: string (nullable = true)
 |-- taker_buy_quote_volume: string (nullable = true)
 |-- ignore: string (nullable = true)



In [ ]:
""""
from pyspark.sql.functions import col
import pyspark.sql.functions as sf
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
data = data.select([col(feature).cast('double') for feature in num_cols])
data= data.select([col("open_time"), sf.timestamp_millis("open_time")])
data.printSchema()
"""


